In [36]:
import nflreadpy
import pandas as pd
import polars as pl

In [37]:
dir(nflreadpy)

['__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'cache',
 'clear_cache',
 'config',
 'downloader',
 'get_current_season',
 'get_current_week',
 'load_combine',
 'load_contracts',
 'load_depth_charts',
 'load_draft_picks',
 'load_ff_opportunity',
 'load_ff_playerids',
 'load_ff_rankings',
 'load_ffverse',
 'load_ftn_charting',
 'load_injuries',
 'load_nextgen_stats',
 'load_officials',
 'load_participation',
 'load_pbp',
 'load_player_stats',
 'load_players',
 'load_rosters',
 'load_rosters_weekly',
 'load_schedules',
 'load_snap_counts',
 'load_stats',
 'load_team_stats',
 'load_teams',
 'load_trades',
 'utils_date',
 'version']

In [38]:
pbp = nflreadpy.load_pbp(2025)

In [39]:
runs = pbp.filter(pl.col('play_type') == 'run')

In [40]:
players = nflreadpy.load_players()

In [43]:
players.columns

['gsis_id',
 'display_name',
 'common_first_name',
 'first_name',
 'last_name',
 'short_name',
 'football_name',
 'suffix',
 'esb_id',
 'nfl_id',
 'pfr_id',
 'pff_id',
 'otc_id',
 'espn_id',
 'smart_id',
 'birth_date',
 'position_group',
 'position',
 'ngs_position_group',
 'ngs_position',
 'height',
 'weight',
 'headshot',
 'college_name',
 'college_conference',
 'jersey_number',
 'rookie_season',
 'last_season',
 'latest_team',
 'status',
 'ngs_status',
 'ngs_status_short_description',
 'years_of_experience',
 'pff_position',
 'pff_status',
 'draft_year',
 'draft_round',
 'draft_pick',
 'draft_team']

In [41]:
players_play_by_play = runs.join(players, left_on='rusher_player_id', right_on='gsis_id', how='left')

In [52]:
stats = players_play_by_play.group_by('rusher_player_id', 'name', 'position').agg([
    pl.col('rushing_yards').sum(),
    pl.col('rush_attempt').count().alias('rush_attempt'),
    pl.col('game_id').n_unique().alias('games_played'),
    pl.col('rush_touchdown').sum().alias('tds')
])
stats = stats.with_columns((pl.col('rushing_yards') / pl.col('rush_attempt')).alias('ypa'))
stats = stats.with_columns((pl.col('tds') / pl.col('rush_attempt')).alias('tds_per_attempt'))

stats = stats.with_columns(
    pl.col('rush_attempt').cast(pl.Int64).neg().rank('dense').cast(pl.Int32).alias('rank')
)
ranked = stats.sort('rush_attempt', descending=True)
with pl.Config(tbl_rows=50, tbl_cols=-1):
    print(ranked.select(['rank', 'name', 'position', 'rush_attempt', 'ypa', 'games_played', 'tds', 'tds_per_attempt']).head(50))

shape: (50, 8)
┌──────┬────────────────┬──────────┬──────────────┬──────────┬──────────────┬─────┬────────────────┐
│ rank ┆ name           ┆ position ┆ rush_attempt ┆ ypa      ┆ games_played ┆ tds ┆ tds_per_attemp │
│ ---  ┆ ---            ┆ ---      ┆ ---          ┆ ---      ┆ ---          ┆ --- ┆ t              │
│ i32  ┆ str            ┆ str      ┆ u32          ┆ f64      ┆ u32          ┆ f64 ┆ ---            │
│      ┆                ┆          ┆              ┆          ┆              ┆     ┆ f64            │
╞══════╪════════════════╪══════════╪══════════════╪══════════╪══════════════╪═════╪════════════════╡
│ 1    ┆ J.Taylor       ┆ RB       ┆ 94           ┆ 5.085106 ┆ 5            ┆ 6.0 ┆ 0.06383        │
│ 2    ┆ C.McCaffrey    ┆ RB       ┆ 91           ┆ 3.098901 ┆ 5            ┆ 0.0 ┆ 0.0            │
│ 3    ┆ J.Cook         ┆ RB       ┆ 90           ┆ 5.0      ┆ 5            ┆ 5.0 ┆ 0.055556       │
│ 4    ┆ A.Jeanty       ┆ RB       ┆ 83           ┆ 4.204819 ┆ 5            

In [ ]:
# build week number, aggregate attempts (count of rush rows) and yards per player-week-week
week_stats = (
    players_play_by_play
    .with_columns(
        week = pl.col("game_id").str.split("_").arr.get(1).cast(pl.Int32)
    )
    .group_by(["rusher_player_id", "name", "position", "week"])
    .agg([
        pl.count().alias("attempts"),
stats = stats.with_columns(
    pl.col('rush_attempt').cast(pl.Int64).neg().rank('dense').cast(pl.Int32).alias('rank')
)
ranked = stats.sort('rush_attempt', descending=True)
with pl.Config(tbl_rows=50, tbl_cols=-1):
    print(ranked.select(['rank', 'name', 'position', 'rush_attempt', 'ypa', 'games_played', 'tds', 'tds_per_attempt']).head(50))

In [53]:
# Test snap_share.py implementation
# Load a small dataset to verify the logic

import sys
sys.path.insert(0, 'f:/Working/ff_pipes')
from snap_share import generate_snap_share_report

# Test with 2024 season
reports = generate_snap_share_report(2024)

# Display sample of results
if reports:
    season_2024 = reports[2024]
    print(f"\nTotal rows: {len(season_2024)}")
    print(f"\nColumns: {season_2024.columns}")
    print(f"\nSample data (first 20 rows):")
    print(season_2024.head(20))
    
    # Check for players with team changes or gaps
    print("\n\nChecking WoW calculation - finding players with NULL first week:")
    first_weeks = season_2024.filter(pl.col('wow_offense_snap_share_change').is_null())
    print(f"Found {len(first_weeks)} rows with NULL WoW (expected for first appearances)")
    print(first_weeks.select(['week', 'player', 'team', 'offense_snap_share', 'wow_offense_snap_share_change']).head(10))


Loading snap count data for seasons: 2024
Loaded 26615 snap count records
Processing seasons: [2024]

Processing season 2024...
  Created grid with 31814 player-week combinations
  Season 2024: 31814 rows in final report

Total rows: 31814

Columns: ['season', 'week', 'player', 'position', 'team', 'opponent', 'offense_snaps', 'offense_snap_share', 'wow_offense_snap_share_change', 'defense_snaps', 'defense_snap_share', 'wow_defense_snap_share_change', 'special_teams_snaps', 'special_teams_snap_share', 'wow_special_teams_snap_share_change']

Sample data (first 20 rows):
shape: (20, 15)
┌────────┬──────┬─────────────┬──────────┬───┬─────────────┬─────────────┬────────────┬────────────┐
│ season ┆ week ┆ player      ┆ position ┆ … ┆ wow_defense ┆ special_tea ┆ special_te ┆ wow_specia │
│ ---    ┆ ---  ┆ ---         ┆ ---      ┆   ┆ _snap_share ┆ ms_snaps    ┆ ams_snap_s ┆ l_teams_sn │
│ i32    ┆ i32  ┆ str         ┆ str      ┆   ┆ _change     ┆ ---         ┆ hare       ┆ ap_share_c │
│    

In [54]:
# Check a specific player's week-over-week progression
# Let's look at a QB who likely played every week
season_2024 = reports[2024]

# Find a player with multiple weeks
player_sample = (
    season_2024
    .filter(pl.col('offense_snap_share') > 0.5)  # Regular starters
    .filter(pl.col('position') == 'QB')
    .group_by('player', 'team')
    .agg(pl.count().alias('weeks'))
    .sort('weeks', descending=True)
    .head(5)
)
print("Sample QBs with most weeks:")
print(player_sample)

# Look at one player's full progression
if len(player_sample) > 0:
    sample_player = player_sample[0, 'player']
    sample_team = player_sample[0, 'team']
    
    print(f"\n\nWeek-by-week for {sample_player} ({sample_team}):")
    player_weeks = (
        season_2024
        .filter(pl.col('player') == sample_player)
        .filter(pl.col('team') == sample_team)
        .select([
            'week', 
            'team',
            'offense_snaps', 
            'offense_snap_share', 
            'wow_offense_snap_share_change'
        ])
        .sort('week')
    )
    print(player_weeks)

Sample QBs with most weeks:
shape: (5, 3)
┌──────────────────┬──────┬───────┐
│ player           ┆ team ┆ weeks │
│ ---              ┆ ---  ┆ ---   │
│ str              ┆ str  ┆ u32   │
╞══════════════════╪══════╪═══════╡
│ Lamar Jackson    ┆ BAL  ┆ 19    │
│ Josh Allen       ┆ BUF  ┆ 19    │
│ Patrick Mahomes  ┆ KC   ┆ 19    │
│ Matthew Stafford ┆ LA   ┆ 18    │
│ Justin Herbert   ┆ LAC  ┆ 18    │
└──────────────────┴──────┴───────┘


Week-by-week for Lamar Jackson (BAL):
shape: (20, 5)
┌──────┬──────┬───────────────┬────────────────────┬───────────────────────────────┐
│ week ┆ team ┆ offense_snaps ┆ offense_snap_share ┆ wow_offense_snap_share_change │
│ ---  ┆ ---  ┆ ---           ┆ ---                ┆ ---                           │
│ i32  ┆ str  ┆ f64           ┆ f64                ┆ f64                           │
╞══════╪══════╪═══════════════╪════════════════════╪═══════════════════════════════╡
│ 1    ┆ BAL  ┆ 80.0          ┆ 1.0                ┆ null                         

C:\Users\LL\AppData\Local\Temp\ipykernel_4308\3535693987.py:11: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  .agg(pl.count().alias('weeks'))
